In [ ]:
import arcpy
from arcpy import env
import os
import numpy as np
from arcgis import GIS
from arcgis.features import GeoAccessor
from arcgis.features import GeoSeriesAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.pivot_table(df, values='a', index='b', columns='c', aggfunc='sum', fill_value=0)
# pd.DataFrame.spatial.from_featureclass(???)  
# df.spatial.to_featureclass(location=???,sanitize_columns=False)  

# gsa = arcgis.features.GeoSeriesAccessor(df['SHAPE'])  
# df['AREA'] = gsa.area  # KNOW YOUR UNITS

In [ ]:
# fill NA values in Spatially enabled dataframes (ignores SHAPE column)
def fill_na_sedf(df_with_shape_column, fill_value=0):
    if 'SHAPE' in list(df_with_shape_column.columns):
        df = df_with_shape_column.copy()
        shape_column = df['SHAPE'].copy()
        del df['SHAPE']
        return df.fillna(fill_value).merge(shape_column,left_index=True, right_index=True, how='inner')
    else:
        raise Exception("Dataframe does not include 'SHAPE' column")

In [ ]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "scratch.gdb", 'hui_for_web.gdb']
gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs[0], outputs[2])

In [ ]:
# hui= r'.\inputs\housing_unit_inventory_2022.gdb\housing_unit_inventory_2022'
hui_wfrc = r".\inputs\WFRC_Housing_Unit_Inventory_20240725.gdb\housing_unit_inventory_2022"
hui_mag = r'.\inputs\MAG_Housing_Unit_Inventory_20240819.gdb\housing_unit_inventory'
hui_box_elder = r".\inputs\WFRC_Housing_Unit_Inventory_20240725.gdb\housing_unit_inventory_box_elder_2020"
hui_tooele = r".\inputs\housing_unit_inventory_tooele_20241007.gdb\housing_unit_inventory_2024"
t = r'.\inputs\transit_stations_and_interchanges.shp'
t_lyr = arcpy.MakeFeatureLayer_management(t, 't_lyr')
parks = r".\inputs\wcv_parks.shp"
trails = r".\inputs\TrailsAndPathways_WFRCMAG.shp"
trails_lyr = arcpy.MakeFeatureLayer_management(trails, 'trails_lyr')
centers = r"E:\Data\Boundaries\WC2050Centers.shp"

In [ ]:
# script arguments
rerun_proximity_functions = False # enable this is proximity has recently been run

In [ ]:
# merge housing datasets, simplify and recalculate UNIT_ID (some units may disappear)
hui_merged = arcpy.management.Merge([hui_wfrc, hui_mag, hui_box_elder, hui_tooele], os.path.join(gdb, 'merged_hui')) 
hui = arcpy.cartography.SimplifyPolygon(hui_merged, os.path.join(gdb, 'simplified_hui'), "POINT_REMOVE", 2.5)
arcpy.CalculateField_management(hui,"UNIT_ID",'!OBJECTID!')

In [ ]:
# use spatial join to summarize get the center name
target_features = hui
join_features = centers
output_features = os.path.join(gdb, "_00_hui_center_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

fields = ['AreaName']
for f in fields:

# field
    fieldindex = fieldmappings.findFieldMapIndex(f)
    fieldmap = fieldmappings.getFieldMap(fieldindex)
    fieldmap.mergeRule = 'first'
    fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join
sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_COMMON", 
                           fieldmappings, "HAVE_THEIR_CENTER_IN")

sj_df = pd.DataFrame.spatial.from_featureclass(sj[0])
sj_df = sj_df[['UNIT_ID', 'AreaName', 'AreaType']].copy()
sj_df.columns = ['UNIT_ID', 'CENTER', 'CENTERTYPE']
sj_df['UNIT_ID'] = sj_df['UNIT_ID'].astype('Int64')

In [ ]:
name = 'parks'
if rerun_proximity_functions == True:
    copy = arcpy.conversion.ExportFeatures(hui, os.path.join(gdb, f'hui_near_{name}'))
    near_result = arcpy.analysis.Near(in_features=copy, near_features=parks, method='GEODESIC')
    df_parks = pd.DataFrame.spatial.from_featureclass(near_result[0])
df_parks = pd.DataFrame.spatial.from_featureclass(os.path.join(gdb, f'hui_near_{name}'))
df_parks['UNIT_ID'] = df_parks['UNIT_ID'].astype('Int64')
# df_parks.head()

In [ ]:
name = 'trails'
if rerun_proximity_functions == True:
    copy = arcpy.conversion.ExportFeatures(hui, os.path.join(gdb, f'hui_near_{name}'))
    arcpy.SelectLayerByAttribute_management(trails_lyr, 'NEW_SELECTION', """Status IN ('EXISTING', 'Existing', 'Current')""")
    near_result = arcpy.analysis.Near(in_features=copy, near_features=trails_lyr, method='GEODESIC')
    df_trails = pd.DataFrame.spatial.from_featureclass(near_result[0])
df_trails = pd.DataFrame.spatial.from_featureclass(os.path.join(gdb, f'hui_near_{name}'))
df_trails['UNIT_ID'] = df_trails['UNIT_ID'].astype('Int64')
# df_trails.head()

In [ ]:
name = 'frontrunner'
if rerun_proximity_functions == True:
    copy = arcpy.conversion.ExportFeatures(hui, os.path.join(gdb, f'hui_near_{name}'))
    arcpy.SelectLayerByAttribute_management(t_lyr, 'NEW_SELECTION', "SubMode = 'Commuter Rail Station' AND Status = 'Current'")
    near_result = arcpy.analysis.Near(in_features=copy, near_features=t_lyr, method='GEODESIC')
    df_fr = pd.DataFrame.spatial.from_featureclass(near_result[0])
df_fr = pd.DataFrame.spatial.from_featureclass(os.path.join(gdb, f'hui_near_{name}'))
df_fr['UNIT_ID'] = df_fr['UNIT_ID'].astype('Int64')
# df_fr.head()

In [ ]:
name = 'lightrail'
if rerun_proximity_functions == True:
    copy = arcpy.conversion.ExportFeatures(hui, os.path.join(gdb, f'hui_near_{name}'))
    arcpy.SelectLayerByAttribute_management(t_lyr, 'NEW_SELECTION', "SubMode = 'Light Rail Station' AND Status = 'Current'")
    near_result = arcpy.analysis.Near(in_features=copy, near_features=t_lyr, method='GEODESIC')
    df_lr = pd.DataFrame.spatial.from_featureclass(near_result[0])
df_lr = pd.DataFrame.spatial.from_featureclass(os.path.join(gdb, f'hui_near_{name}'))
df_lr['UNIT_ID'] = df_lr['UNIT_ID'].astype('Int64')
# df_lr.head()

In [ ]:
name = 'brt'
if rerun_proximity_functions == True:
    copy = arcpy.conversion.ExportFeatures(hui, os.path.join(gdb, f'hui_near_{name}'))
    arcpy.SelectLayerByAttribute_management(t_lyr, 'NEW_SELECTION', "SubMode = 'BRT Stop' And (Status = 'Current' Or Status IS NULL)")
    near_result = arcpy.analysis.Near(in_features=copy, near_features=t_lyr, method='GEODESIC')
    df_brt = pd.DataFrame.spatial.from_featureclass(near_result[0])
df_brt = pd.DataFrame.spatial.from_featureclass(os.path.join(gdb, f'hui_near_{name}'))
df_brt['UNIT_ID'] = df_brt['UNIT_ID'].astype('Int64')
# df_brt.head()

In [ ]:
name = 'fwyexit'
if rerun_proximity_functions == True:
    copy = arcpy.conversion.ExportFeatures(hui, os.path.join(gdb, f'hui_near_{name}'))
    arcpy.SelectLayerByAttribute_management(t_lyr, 'NEW_SELECTION', "SubMode = 'Interchange' AND Status = 'Current'")
    near_result = arcpy.analysis.Near(in_features=copy, near_features=t_lyr, method='GEODESIC')
    df_fwy = pd.DataFrame.spatial.from_featureclass(near_result[0])
df_fwy = pd.DataFrame.spatial.from_featureclass(os.path.join(gdb, f'hui_near_{name}'))
df_fwy['UNIT_ID'] = df_fwy['UNIT_ID'].astype('Int64')
# df_fwy.head()

In [ ]:
# convert distance from meters to miles
df_fr['DIST_FR'] = df_fr['NEAR_DIST']*.000621371
df_lr['DIST_LR'] = df_lr['NEAR_DIST']*.000621371
df_brt['DIST_BRT'] = df_brt['NEAR_DIST']*.000621371
df_fwy['DIST_FWYE'] = df_fwy['NEAR_DIST']*.000621371
df_parks['DIST_PARK'] = df_parks['NEAR_DIST']*.000621371
df_trails['DIST_TRAIL'] = df_trails['NEAR_DIST']*.000621371

df_fr['DIST_FR'] = round(df_fr['DIST_FR'], 2)
df_lr['DIST_LR'] = round(df_lr['DIST_LR'], 2)
df_brt['DIST_BRT'] = round(df_brt['DIST_BRT'], 2)
df_fwy['DIST_FWYE'] = round(df_fwy['DIST_FWYE'], 2)
df_parks['DIST_PARK'] = round(df_parks['DIST_PARK'], 2)
df_trails['DIST_TRAIL'] = round(df_trails['DIST_TRAIL'], 2)

df_fr = df_fr[['UNIT_ID', 'DIST_FR']].copy()
df_lr = df_lr[['UNIT_ID', 'DIST_LR']].copy()
df_brt = df_brt[['UNIT_ID', 'DIST_BRT']].copy()
df_fwy = df_fwy [['UNIT_ID', 'DIST_FWYE']].copy()
df_parks = df_parks[['UNIT_ID', 'DIST_PARK']].copy()
df_trails = df_trails[['UNIT_ID', 'DIST_TRAIL']].copy()

In [ ]:
df_lr['UNIT_ID'].value_counts()

In [ ]:
# merge dataframes
hui_df = pd.DataFrame.spatial.from_featureclass(hui[0])
hui_df = (hui_df.merge(sj_df, on='UNIT_ID', how='left') 
                .merge(df_fr, on='UNIT_ID', how='left') 
                .merge(df_lr, on='UNIT_ID', how='left')  
                .merge(df_brt, on='UNIT_ID', how='left')
                .merge(df_fwy, on='UNIT_ID', how='left')
                .merge(df_parks, on='UNIT_ID', how='left')
                .merge(df_trails, on='UNIT_ID', how='left')) 

# clear memory
# for df in [sj_df, df_fr, df_lr, df_brt, df_fwy, df_parks, df_trails]: del df

In [ ]:
hui_df[['APX_BLT_YR','BLT_YR2']].to_csv('test.csv')

In [ ]:
# create the date field for timeslider
hui_df['APX_BLT_YR'] = hui_df['APX_BLT_YR'].astype('Int64')
# hui_df['BLT_YR2'] = pd.to_datetime(hui_df['APX_BLT_YR'], format='%Y',  errors='coerce')
hui_df.loc[hui_df['APX_BLT_YR'].isna() == True, 'APX_BLT_YR'] = 0
hui_df['BLT_YR2'] = pd.to_datetime(hui_df['APX_BLT_YR'], format='%Y',  errors='coerce')
# hui_df.loc[hui_df['APX_BLT_YR'] > 0, 'BLT_YR2'] = pd.to_datetime(hui_df['APX_BLT_YR'], format='%Y',  errors='coerce')

# alter the built decade attribute
hui_df.loc[(hui_df['APX_BLT_YR'] >= 1840) & (hui_df['APX_BLT_YR'] < 1850), 'BLT_DECADE'] = "1840"
hui_df.loc[(hui_df['APX_BLT_YR'] >= 1850) & (hui_df['APX_BLT_YR'] < 1860), 'BLT_DECADE'] = "1850"
hui_df.loc[(hui_df['APX_BLT_YR'] >= 1860) & (hui_df['APX_BLT_YR'] < 1870), 'BLT_DECADE'] = "1860"
hui_df.loc[(hui_df['APX_BLT_YR'] >= 1870) & (hui_df['APX_BLT_YR'] < 1880), 'BLT_DECADE'] = "1870"
hui_df.loc[(hui_df['APX_BLT_YR'] >= 1880) & (hui_df['APX_BLT_YR'] < 1890), 'BLT_DECADE'] = "1880"
hui_df.loc[(hui_df['APX_BLT_YR'] >= 1890) & (hui_df['APX_BLT_YR'] < 1900), 'BLT_DECADE'] = "1890"
hui_df.loc[(hui_df['APX_BLT_YR'] >= 1900) & (hui_df['APX_BLT_YR'] < 1910), 'BLT_DECADE'] = "1900"
hui_df.loc[(hui_df['APX_BLT_YR'] >= 1910) & (hui_df['APX_BLT_YR'] < 1920), 'BLT_DECADE'] = "1910"
hui_df.loc[(hui_df['APX_BLT_YR'] >= 1920) & (hui_df['APX_BLT_YR'] < 1930), 'BLT_DECADE'] = "1920"
hui_df.loc[(hui_df['APX_BLT_YR'] >= 1930) & (hui_df['APX_BLT_YR'] < 1940), 'BLT_DECADE'] = "1930"
hui_df.loc[(hui_df['APX_BLT_YR'] >= 1940) & (hui_df['APX_BLT_YR'] < 1950), 'BLT_DECADE'] = "1940"
hui_df.loc[(hui_df['APX_BLT_YR'] >= 1950) & (hui_df['APX_BLT_YR'] < 1960), 'BLT_DECADE'] = "1950"
hui_df.loc[(hui_df['APX_BLT_YR'] >= 1960) & (hui_df['APX_BLT_YR'] < 1970), 'BLT_DECADE'] = "1960"
hui_df.loc[(hui_df['APX_BLT_YR'] >= 1970) & (hui_df['APX_BLT_YR'] < 1980), 'BLT_DECADE'] = "1970"
hui_df.loc[(hui_df['APX_BLT_YR'] >= 1980) & (hui_df['APX_BLT_YR'] < 1990), 'BLT_DECADE'] = "1980"
hui_df.loc[(hui_df['APX_BLT_YR'] >= 1990) & (hui_df['APX_BLT_YR'] < 2000), 'BLT_DECADE'] = "1990"
hui_df.loc[(hui_df['APX_BLT_YR'] >= 2000) & (hui_df['APX_BLT_YR'] < 2010), 'BLT_DECADE'] = "2000"
hui_df.loc[(hui_df['APX_BLT_YR'] >= 2010) & (hui_df['APX_BLT_YR'] < 2020), 'BLT_DECADE'] = "2010"
hui_df.loc[(hui_df['APX_BLT_YR'] >= 2020) & (hui_df['APX_BLT_YR'] < 2030), 'BLT_DECADE'] = "2020"
hui_df.loc[hui_df['BLT_DECADE'].isin(['1840', '1850', '1860', '1870', '1880', '1890']) == True, 'BLT_DECADE'] = '1840-1890'
hui_df.loc[hui_df['BLT_DECADE'].isin(['1900', '1910', '1920', '1930', '1940', '1950']) == True, 'BLT_DECADE'] = '1900-1950'

hui_df.loc[hui_df['SUBTYPE'].isin(['single_family', 'single_family_adu']) == True, 'TYPE'] = 'single_family'
hui_df.loc[hui_df['SUBTYPE'].isin(['single_family', 'single_family_adu']) == False, 'TYPE'] = 'multi_family'

In [ ]:
# MAG has some data from 2025; remove
hui_df = hui_df[hui_df['APX_BLT_YR'] < 2025].copy()

# Remove MAG EVAL_DATE field
del hui_df['EVAL_DATE']

# delete fields added from simplify polygon tool
del hui_df['InPoly_FID']
del hui_df['MaxSimpTol']
del hui_df['MinSimpTol']
hui_df.columns

In [ ]:
print(hui_df.shape)

In [ ]:
# export polygon file
hui_df.spatial.to_featureclass(location=os.path.join(gdb2, 'hui_web_version'),sanitize_columns=False)

In [ ]:
# export points file
arcpy.management.FeatureToPoint(os.path.join(gdb2, 'hui_web_version'), os.path.join(gdb2, 'hui_pts_web_version'), "INSIDE")

# exploration